In [1]:
# -*- coding: utf-8 -*-
import warnings
import os.path
import ow_f01, ow_train, ow_predict, predict_mean
import pickle
from code.refactor.common import loadSettingsFromYamlFile,save_object
import pandas as pd
import numpy as np
import datetime
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
warnings.filterwarnings("ignore", category=DeprecationWarning)


/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
path = 'tmp/data/shishang/'
result_path = 'tmp/data/shishang/test_0705'
suffix = '.da'
item = 'p1'
for_what = ['train', 'predict']
cate = 2584

scenarioSettingsPath = 'code/refactor/ow_scenario.yaml'
scenario = loadSettingsFromYamlFile(scenarioSettingsPath)
area_rdc_map = pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/area_rdc_mapping.csv')
holidays_df=pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/holidays.csv')


In [3]:
def reduce_df_mem_usage(df):
    # memery now
    start_mem_usg = df.memory_usage().sum() / 1024 ** 2
    print("Memory usage of the dataframe is :", start_mem_usg, "MB")
    
    #np.nan will be handled as float
    NAlist = []
    for col in df.columns:
        # filter object type
        if (df[col].dtypes == np.float64):
            df[col] = df[col].astype(np.float32)
            continue
        if (df[col].dtypes != object)&(df[col].dtypes != 'datetime64[ns]'):
            
            print("**************************")
            print("columns: %s"%col)
            print("dtype before: %s"%df[col].dtype)
            
            # if int or not
            isInt = False
            mmax = df[col].max()
            mmin = df[col].min()
            
            # Integer does not support NA, therefore Na needs to be filled
            if not np.isfinite(df[col]).all():
                NAlist.append(col)
                #continue
                df[col].fillna(-999, inplace=True) # fill -999
                
            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = np.fabs(df[col] - asint)
            result = result.sum()
            if result < 0.01: # absolute error < 0.01,then could be saw as integer
                isInt = True
            
            # make interger / unsigned Integer datatypes
            if isInt:
                if mmin >= 0: # min>=0, then unsigned integer
                    if mmax <= np.iinfo(np.uint8).max:
                        df[col] = df[col].astype(np.uint8)
                    elif mmax <= np.iinfo(np.uint16).max:
                        df[col] = df[col].astype(np.uint16)
                    elif mmax <= np.iinfo(np.uint32).max:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mmin > np.iinfo(np.int8).min and mmax < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mmin > np.iinfo(np.int16).min and mmax < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mmin > np.iinfo(np.int32).min and mmax < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mmin > np.iinfo(np.int64).min and mmax < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)  
            df.replace(-999, np.nan, inplace=True)
            print("dtype after: %s"%df[col].dtype)
            print("********************************")
    print("___MEMORY USAGE AFTER CONVERSION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df, NAlist

In [4]:
p1_used_header=['productkey', 'promotionkey', 'startdatetime', 'enddatetime', 'jdprice', 'syntheticgrossprice', 'promotiondesc', 'promotiondesc_flag', 'promotiontype', 'promotionsubtype',
                'areatypearray', 'tokenflag', 'directdiscount_discount', 'directdiscount_availabilitynumber', 'bundle_subtype1_threshold', 'bundle_subtype1_giveaway',
                'bundle_subtype4_threshold1', 'bundle_subtype4_giveaway1', 'bundle_subtype4_threshold2', 'bundle_subtype4_giveaway2', 'bundle_subtype4_threshold3',
                'bundle_subtype4_giveaway3', 'bundle_subtype2_threshold', 'bundle_subtype2_giveaway', 'bundle_subtype2_maximumgiveaway', 'bundle_subtype15_thresholdnumber1',
                'bundle_subtype15_giveawayrate1', 'bundle_subtype15_thresholdnumber2', 'bundle_subtype15_giveawayrate2', 'bundle_subtype15_thresholdnumber3',
                'bundle_subtype15_giveawayrate3', 'bundle_subtype6_thresholdnumber', 'bundle_subtype6_freenumber', 'suit_maxvaluepool', 'suit_minvaluepool', 'suit_avgvaluepool',
                'suit_discount', 'directdiscount_saleprice', 'bundle_subtype1_percent', 'bundle_subtype4_percent', 'bundle_subtype2_percent', 'bundle_subtype15_percent',
                'bundle_subtype6_percent', 'suit_percent', 'allpercentdiscount', 'mainproductkey', 'hierarchylevel3key', 'createdate', 'statuscode', 'dt']


In [5]:
'''
	hyperparameters
'''
cate = 2584
pred_date = pd.to_datetime('2018-05-07')
scenario['lookforwardPeriodDays'] = 7


In [44]:
p1_path = os.path.join(path, str(cate)+'/'+str(cate)+'_'+item+suffix)
period_promo_raw = pd.read_csv(p1_path,sep='\t',header=None)
period_promo_raw.columns=p1_used_header
    
for fw in for_what:
    print "output and save: %s_p1_%s"%(str(cate),fw)
    period_promo_raw_clean = period_promo_raw.copy()
    train_pred_gate = fw   # 'train'
    f01_output = ow_f01.generate_f01_promo(area_rdc_map, period_promo_raw_clean,scenario, train_pred_gate, ForecastStartDate=pred_date)
    f01_output.to_csv(os.path.join(result_path, train_pred_gate+'_'+item+'_'+str(cate)+'.csv'),index=False)


output and save: 2584_p1_train
convert to Float columns:
allpercentdiscount
convert to Float columns:
bundle_subtype15_giveawayrate1
convert to Float columns:
bundle_subtype15_giveawayrate2
convert to Float columns:
bundle_subtype15_giveawayrate3
convert to Float columns:
bundle_subtype15_percent
convert to Float columns:
bundle_subtype15_thresholdnumber1
convert to Float columns:
bundle_subtype15_thresholdnumber2
convert to Float columns:
bundle_subtype15_thresholdnumber3
convert to Float columns:
bundle_subtype1_giveaway
convert to Float columns:
bundle_subtype1_percent
convert to Float columns:
bundle_subtype1_threshold
convert to Float columns:
bundle_subtype2_giveaway
convert to Float columns:
bundle_subtype2_maximumgiveaway
convert to Float columns:
bundle_subtype2_percent
convert to Float columns:
bundle_subtype2_threshold
convert to Float columns:
bundle_subtype4_giveaway1
convert to Float columns:
bundle_subtype4_giveaway2
convert to Float columns:
bundle_subtype4_giveaway3
co

In [6]:
seasonality_df = pd.read_csv('tmp/data/870_season.csv', parse_dates=['Date'])
file_path = os.path.join(path, str(cate)+'/'+str(cate)+'_'+'ts.da')
ts_df = pd.read_csv(file_path,header=None,sep='\t')

pro_canlender_path = os.path.join(path, str(cate)+'/'+str(cate)+'_'+'p2.da')
promoCalendarDf = pd.read_csv(pro_canlender_path,sep='\t',header=None)
promoCalendarDf.columns=['ProductKey', 'Date', 'HierarchyLevel3Key', 'PromotionCount', 'bundlecount', 'MaxDiscount', 'MinDiscount', 'AvgDiscount', 'MaxSyntheticDiscountA',
				         'MinSyntheticDiscountA', 'AvgSyntheticDiscountA', 'MaxBundleDiscount', 'MinBundleDiscount', 'AvgBundleDiscount', 'MaxDirectDiscount', 'MinDirectDiscount',
				         'AvgDirectDiscount', 'MaxFreegiftDiscount', 'MinFreegiftDiscount', 'AvgFreegiftDiscount', 'SyntheticGrossPrice', 'promotionkey', 'promotiontype',
				         'promotionsubtype', 'syntheticgrossprice_vb', 'jdprice', 'syntheticdiscounta_vb', 'durationinhours', 'daynumberinpromotion', 'bundleflag', 'directdiscountflag',
				         'freegiftflag', 'suitflag', 'numberproducts', 'numberhierarchylevel1', 'numberhierarchylevel2', 'numberhierarchylevel3', 'strongmark', 'stockprice', 'dt']


In [7]:
p1_out_path = os.path.join(result_path, 'train'+'_p1_'+str(cate)+'.csv')
period_promo_raw = pd.read_csv(p1_out_path,parse_dates=['Date'])

seasonality_df_train = seasonality_df.copy()
ts_df_train = ts_df.copy()
promoCalendarDf_train = promoCalendarDf.copy()

model,feature=ow_train.train(area_rdc_map,period_promo_raw,promoCalendarDf_train,ts_df_train,scenario,holidays_df,seasonality_df_train,process_f01_flag=False,mode='dev',ForecastStartDate=pred_date)


convert to Int columns:
Holiday
convert to Int columns:
Ind_1111_pre
convert to Int columns:
Ind_1111
convert to Int columns:
Ind_1111_post
convert to Int columns:
Ind_618_pre
convert to Int columns:
Ind_618
convert to Int columns:
Ind_618_post
convert to Int columns:
Ind_1212
convert to Float columns:
MaxSyntheticDiscountA
convert to Float columns:
MinSyntheticDiscountA
convert to Float columns:
AvgSyntheticDiscountA
convert to Float columns:
SyntheticGrossPrice
convert to Float columns:
promotionkey
convert to Float columns:
promotiontype
convert to Float columns:
promotionsubtype
convert to Float columns:
syntheticgrossprice_vb
convert to Float columns:
jdprice
convert to Float columns:
syntheticdiscounta_vb
convert to Float columns:
durationinhours
convert to Float columns:
daynumberinpromotion
convert to Float columns:
bundleflag
convert to Float columns:
directdiscountflag
convert to Float columns:
freegiftflag
convert to Float columns:
suitflag
convert to Float columns:
stockpri

ow_train.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ts_df_pred.replace('null', np.nan, inplace=True)
ow_train.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ts_df_pred.replace(-999, np.nan, inplace=True)


use cart feature?
False
| Add date features...
| - Add month of the year...
| - Add day of the week...
| Calculating national rolling features...
('| - Rolling value:', 'xxxHashColumn')
| - Rolling 14 mean...
| - Rolling 7 mean...
| - Rolling 5 mean...
| - Rolling 3 mean...
| - Rolling 2 mean...
| - Rolling 1 mean...
| - Rolling 14 median...
| - Rolling 7 median...
| - Rolling decay 28 mean...
| - Rolling decay 14 mean...
| - Rolling decay 7 mean...
| - Rolling decay 3 mean...
| - Added rolling features in 556.928106 seconds 

| Calculating  rolling features...
('| - Rolling value:', 'OrderNonOutlierVolume')
| - Rolling 360 mean...
| - Rolling 180 mean...
| - Rolling 90 mean...
| - Rolling 28 mean...
| - Rolling 14 mean...
| - Rolling 7 mean...
| - Rolling 5 mean...
| - Rolling 3 mean...
| - Rolling 2 mean...
| - Rolling 1 mean...
| - Rolling 14 median...
| - Rolling 7 median...
| - Rolling decay 360 mean...
| - Rolling decay 180 mean...
| - Rolling decay 90 mean...
| - Rolling decay 2

| - Added rolling features in 2603.361878 seconds 

| Calculating volume rolling features...
('| - Rolling value:', 'salesForecast')
| - Shift 365...
| - Shift 180...
| - Shift 90...
| - Shift 30...
| - Shift 7...
| - Added rolling features in 250.636723 seconds 

featuresDf shape
(15902399, 141)
rdc
3
rdc
4
rdc
5
rdc
6
rdc
9
rdc
10
rdc
316
rdc
772


In [8]:
feature.to_csv(os.path.join(result_path, str(cate)+'_train_feature.csv'),index=False)

In [9]:
save_object(model, os.path.join(result_path, str(cate)+'_train_model.pkl'))

In [12]:
feature.Date.min()

Timestamp('2016-04-18 00:00:00')

In [11]:
feature.shape

(15902399, 141)

In [33]:
pred_date

Timestamp('2018-02-01 00:00:00')

In [14]:
p1_out_path = os.path.join(result_path, 'predict'+'_p1_'+str(cate)+'.csv')
period_promo_raw = pd.read_csv(p1_out_path,parse_dates=['Date'])

seasonality_df_test = seasonality_df.copy()
ts_df_test = ts_df.copy()
promoCalendarDf_test = promoCalendarDf.copy()

# model_path = os.path.join(result_path, str(cate)+'/'+str(cate)+'_train_model.pkl')
# with open(model_path,'r') as input:
#     model = pickle.load(input)
q_pred_result,df_fut=ow_predict.predict(area_rdc_map,period_promo_raw,promoCalendarDf_test,ts_df_test,scenario,holidays_df,model,seasonality_df_test,process_f01_flag=False,mode='dev',ForecastStartDate=pred_date)


ow_predict.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ts_df.replace('null', np.nan, inplace=True)
ow_predict.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ts_df.replace(-999, np.nan, inplace=True)
ow_predict.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ts_df.replace('None', np.nan, inplace=True)


convert to Int columns:
Holiday
convert to Int columns:
Ind_1111_pre
convert to Int columns:
Ind_1111
convert to Int columns:
Ind_1111_post
convert to Int columns:
Ind_618_pre
convert to Int columns:
Ind_618
convert to Int columns:
Ind_618_post
convert to Int columns:
Ind_1212
convert to Float columns:
MaxSyntheticDiscountA
convert to Float columns:
MinSyntheticDiscountA
convert to Float columns:
AvgSyntheticDiscountA
convert to Float columns:
SyntheticGrossPrice
convert to Float columns:
promotionkey
convert to Float columns:
promotiontype
convert to Float columns:
promotionsubtype
convert to Float columns:
syntheticgrossprice_vb
convert to Float columns:
jdprice
convert to Float columns:
syntheticdiscounta_vb
convert to Float columns:
durationinhours
convert to Float columns:
daynumberinpromotion
convert to Float columns:
bundleflag
convert to Float columns:
directdiscountflag
convert to Float columns:
freegiftflag
convert to Float columns:
suitflag
convert to Float columns:
stockpri

code/refactor/common.py:1363: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  + df_past[similarLevels[2]]
code/refactor/common.py:1364: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_past['SyntheticPromotionSubType'] = df_past['SyntheticPromotionSubType'].fillna(0.0)


| - group by SyntheticPromotionSubType...


ow_predict.py:454: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  pred_df['ypred'] =ypred
ow_predict.py:455: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  pred_df['RDCKey'] = rdc
/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.

In [17]:
p1_out_path = os.path.join(result_path, 'predict'+'_p1_'+str(cate)+'.csv')
period_promo_raw = pd.read_csv(p1_out_path,parse_dates=['Date'])

seasonality_df_mean = seasonality_df.copy()
ts_df_mean = ts_df.copy()
promoCalendarDf_mean = promoCalendarDf.copy()

#train_feature_path = os.path.join(result_path, str(cate) + '/' + str(cate) + '_train_feature.csv')
train_feature_df = feature
q_mean_result,df_fut_mean=predict_mean.predict(area_rdc_map,period_promo_raw,promoCalendarDf,ts_df,scenario,holidays_df,model,seasonality_df,process_f01_flag=False,mode='dev',ForecastStartDate=pred_date,train_feature=train_feature_df)


predict_mean.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ts_df.replace('null', np.nan, inplace=True)
predict_mean.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ts_df.replace(-999, np.nan, inplace=True)
predict_mean.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ts_df.replace('None', np.nan, inplace=True)


convert to Float columns:
RDCKey
convert to Float columns:
ProductKey
convert to Float columns:
HierarchyLevel1Key
convert to Float columns:
HierarchyLevel2Key
convert to Float columns:
HierarchyLevel3Key
convert to Float columns:
brand_code
convert to Float columns:
sales
convert to Float columns:
priceAfterDiscount
convert to Float columns:
jd_prc
convert to Float columns:
vendibility
convert to Float columns:
counterState
convert to Float columns:
salesForecast
convert to Float columns:
reserveState
convert to Float columns:
stockQuantity
convert to Float columns:
utc_flag
convert to Int columns:
Holiday
convert to Int columns:
Ind_1111_pre
convert to Int columns:
Ind_1111
convert to Int columns:
Ind_1111_post
convert to Int columns:
Ind_618_pre
convert to Int columns:
Ind_618
convert to Int columns:
Ind_618_post
convert to Int columns:
Ind_1212
convert to Float columns:
MaxSyntheticDiscountA
convert to Float columns:
MinSyntheticDiscountA
convert to Float columns:
AvgSyntheticDisco

predict_mean.py:450: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  pred_df['ypred'] =ypred
predict_mean.py:451: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  pred_df['RDCKey'] = rdc


In [15]:
real_ts_df = ts_df.copy()
real_ts_df.columns = ['Date', 'ind', 'RDCKey', 'ProductKey', 'HierarchyLevel1Key', 'HierarchyLevel2Key', 'HierarchyLevel3Key', 'brand_code', 'sales', 'priceAfterDiscount', 'jd_prc', 'vendibility', 'counterState', 'salesForecast', 'reserveState', 'stockQuantity', 'utc_flag']
real_ts_df['Date'] = pd.to_datetime(real_ts_df['Date'])
simplified_ts_df = real_ts_df[real_ts_df.Date.between('2018-05-07','2018-05-13')]


In [17]:
feature.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15902399 entries, 0 to 15902398
Columns: 141 entries, Date to bd_discount_sgp_wgt
dtypes: datetime64[ns](3), float64(121), int64(14), object(3)
memory usage: 16.8+ GB


In [ ]:
train_feature_df = feature
#model_path = 'tmp/data/shishang/' + str(cate) + '/' + str(cate) + '_train_model.pkl'

df_new, NAlist = reduce_df_mem_usage(train_feature_df)

ForecastStartDate = pd.to_datetime(pred_date)
DataStartDate = ForecastStartDate - datetime.timedelta(days=scenario['lookbackPeriodDays'])
PredictEndDate = ForecastStartDate + datetime.timedelta(days=(scenario['lookforwardPeriodDays']-1))


('Memory usage of the dataframe is :', 17228, 'MB')
**************************
columns: HierarchyLevel1Key
dtype before: int64
dtype after: uint16
********************************
**************************
columns: HierarchyLevel2Key
dtype before: int64
dtype after: uint16
********************************
**************************
columns: HierarchyLevel3Key
dtype before: int64
dtype after: uint16
********************************
**************************
columns: brand_code
dtype before: int64
dtype after: uint32
********************************
**************************
columns: utc_flag
dtype before: int64
dtype after: uint8
********************************
**************************
columns: Holiday
dtype before: int64


In [20]:
actual = simplified_ts_df
actual.Date = pd.to_datetime(actual.Date)
actual.RDCKey = actual.RDCKey.astype(float)

list_keys = ['Date','RDCKey','ProductKey']

raw = q_pred_result  ###bottomup forecast
raw = raw[list_keys + ['salesForecast','ypred']]
raw.rename(columns={'ypred':'ypred_raw'},inplace=True)
raw.drop('salesForecast',axis=1,inplace=True)

feat_cols = ['dd_price_weighted','bd_price_weighted','dd_price_weighted_x','bd_price_weighted_x','SyntheticGrossPrice']
mean_df = q_mean_result
mean_df = mean_df[list_keys + feat_cols + ['salesForecast','ypred']]
mean_df.rename(columns={'ypred':'ypred_mean_promo'},inplace=True)
mean_df.drop('salesForecast',axis=1,inplace=True)

new_df = raw.merge(mean_df,on=list_keys)
new_df.Date = pd.to_datetime(new_df.Date)
new_df = pd.merge(new_df, actual[list_keys+['salesForecast']], how='left',on = list_keys)

/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

In [30]:
new_df.head()

,Date,RDCKey,ProductKey,ypred_raw,dd_price_weighted,bd_price_weighted,dd_price_weighted_x,bd_price_weighted_x,SyntheticGrossPrice,ypred_mean_promo,salesForecast
0,2018-02-01,3.0,255778,24.334154,153.427716,152.811594,149.0,149.0,149.0,22.588051,26.0
1,2018-02-02,3.0,255778,24.064211,153.427716,152.811594,149.0,149.0,149.0,21.853262,29.0
2,2018-02-03,3.0,255778,24.064211,153.427716,152.811594,149.0,149.0,149.0,21.853262,27.0
3,2018-02-04,3.0,255778,24.064211,153.427716,152.811594,149.0,149.0,149.0,21.853262,21.0
4,2018-02-05,3.0,255778,23.172569,153.427716,152.811594,149.0,149.0,149.0,22.669336,21.0


In [21]:
exclu_promo_features = ['strongmark','flashsale_ind','dd_ind','bundle_ind','bundle_buy199get100_ind','suit_ind','freegift_ind']
update_cols = list(set(scenario['promo_feature_cols'])- set(exclu_promo_features))
need_cols = ['Date','RDCKey','ProductKey','HierarchyLevel3Key'] + update_cols

uses_promo = ['mean','no']
df = df_new #df_new
reg_cols = []#['Holiday','Ind_1111_pre','Ind_1111','Ind_1111_post','Ind_618_pre','Ind_618','Ind_618_post','Ind_1212','Month','DayOfWeek']
#reg_cols = ['Holiday','Ind_1111_pre','Ind_1111','Ind_1111_post','Ind_618_pre','Ind_618','Ind_618_post','Ind_1212','Month','DayOfWeek']

In [22]:
for use_promo in uses_promo:
    if use_promo == 'mean':
        df1 = df[need_cols]
        groupkeys = ['RDCKey','ProductKey','HierarchyLevel3Key']
        promo_feature_cols =  scenario['promo_feature_cols']
        df11 = df1.groupby(groupkeys)[update_cols].mean().reset_index()
        df2 = pd.merge(df,df11[groupkeys + update_cols], how='left',on=groupkeys)

        rename_update_cols = [col+'_y' for col in update_cols]
        for col in update_cols:
            df2.rename(columns={col+'_y': col},inplace=True)
            df2.drop(col+'_x',axis=1,inplace=True)
        grouped = df2.groupby('RDCKey')
    else:
        #histoty bottomup forecast
        grouped = df.groupby('RDCKey')
    result_list = []
    for rdc, history_df in grouped:
        if rdc in model.keys():
            this_model = model[rdc]
        else:
            continue
        ''' predict model '''
        xColumns = scenario['selectedColumns']['features']

        if 'RDCKey' in xColumns:# 删除季节性,RDCKEY
            xColumns.remove('skuDecomposedTrend')
            xColumns.remove('skuDecomposedSeasonal')
            xColumns.remove('level3DecomposedTrend')
            xColumns.remove('level3DecomposedSeasonal')
            xColumns.remove('Curve')
            xColumns.remove('RDCKey')
        
        X_history = history_df[xColumns]

        history_xtest = xgb.DMatrix(X_history.values, missing=np.NaN )
        ypred = this_model.predict(history_xtest)
        history_df['ypred'] =ypred
        history_df['RDCKey'] = rdc

        ''' Tuning result '''
        lanjie = history_df[(history_df.ypred<0)]
        if len(lanjie)>0:
            history_df.ix[lanjie.index,'ypred'] = 0
        result_list.append(history_df)
    final_result = pd.concat(result_list)
    if use_promo == 'no':
        raw_train_df = final_result[list_keys + reg_cols + scenario['promo_feature_cols'] + ['salesForecast','ypred']]
    else:
        #use_promo == 'mean':
        train_df_mean = final_result[list_keys + reg_cols + scenario['promo_feature_cols'] + ['salesForecast','ypred']]
        train_df_mean.rename(columns={'ypred':'ypred_mean_promo'}, inplace=True)

/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [23]:
raw_train_df = pd.merge(raw_train_df, train_df_mean[list_keys+['ypred_mean_promo']], how='left', on=list_keys)
raw_test_df = q_pred_result
raw_test_df = raw_test_df[list_keys + reg_cols + scenario['promo_feature_cols'] + ['salesForecast','ypred']]
raw_test_df.Date = pd.to_datetime(raw_test_df.Date)

used_cols = reg_cols + ['MaxSyntheticDiscountA']  #['MaxBundleDiscount','MaxDirectDiscount','MaxDiscount','MaxSyntheticDiscountA','daynumberinpromotion','PromotionCount']
raw_train_df.Date = pd.to_datetime(raw_train_df.Date)
raw_train_df = raw_train_df[raw_train_df.Date < pred_date]
raw_train_df = raw_train_df[list_keys + reg_cols + scenario['promo_feature_cols'] + ['ypred','ypred_mean_promo']]
input_df = pd.concat([raw_train_df, raw_test_df])

for col in used_cols:
    #input_df[col] = input_df.groupby(['RDCKey','ProductKey'])[col].transform(lambda x: x.fillna(method='bfill').fillna(0))
    #input_df[col] = input_df.groupby(['RDCKey','ProductKey'])[col].transform(lambda x: x.fillna(method='ffill').fillna(0))
    #input_df = input_df[input_df['MaxSyntheticDiscountA'].between(-1,1)]
    
    input_df = input_df[~(input_df[col].isnull())]
    input_df = input_df[input_df[col].between(-1,1)]

In [24]:
value_type = 'ypred_mean_promo'
final_df = pd.DataFrame()
a = 1
grouped = input_df.groupby(['RDCKey','ProductKey'])
for (rdc, sku), group in grouped:
    if group.Date.min() < ForecastStartDate and group.Date.max() >= ForecastStartDate:
        print a
        a = a + 1
        train_df = group[group.Date < ForecastStartDate]
        test_df = group[group.Date >= ForecastStartDate]
        x_train_df = train_df[used_cols]
        x_test_df = test_df[used_cols]
        
        y_train = train_df['ypred'] - train_df[value_type]
        y_test = test_df['salesForecast']
        lm = LinearRegression()
        lm.fit(x_train_df, y_train)
        Intercept = lm.intercept_
        RSquare = lm.score(x_train_df, y_train)
        lm_predict_result = lm.predict(x_test_df)
        test_result = pd.DataFrame()
        for col in list_keys+['salesForecast','ypred']:
            test_result[col] = test_df[col]
        test_result['reg_result'] = lm_predict_result

        ###gaussian###
        '''
        if len(x_train_df[used_cols].drop_duplicates()) == 1:
            test_result = pd.DataFrame()
            for col in list_keys+['salesForecast','ypred']:
                test_result[col] = test_df[col]
            test_result['reg_result'] = y_train.tail().mean()
        else:
            lm = make_pipeline(GaussianFeatures(5), Lasso(alpha=0.1))
            lm.fit(np.array(x_train_df), np.array(y_train))
            Intercept = lm.steps[1][1].intercept_
            RSquare = lm.score(np.array(x_train_df), np.array(y_train))
            lm_predict_result = lm.predict(np.array(x_test_df))
            test_result = pd.DataFrame()
            for col in list_keys+['salesForecast','ypred']:
                test_result[col] = test_df[col]
            test_result['reg_result'] = lm_predict_result
        '''
        ###polynomial###
        '''
        pf = PolynomialFeatures(degree=2)
        pModel = LinearRegression()
        pModel.fit(pf.fit_transform(x_train_df), y_train)
        pf_predict_result = pModel.predict(pf.fit_transform(x_test_df))
        test_result = pd.DataFrame()
        for col in list_keys+['salesForecast','ypred']:
            test_result[col] = test_df[col]
        test_result['reg_result'] = pf_predict_result
        '''
        final_df = pd.concat([final_df, test_result])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [25]:
mean_final = final_df
mean_final.rename(columns={'reg_result':'mean_promo_reg_result'},inplace=True)
final_df.drop('salesForecast',axis=1,inplace=True)
final_df.drop('ypred',axis=1,inplace=True)
new_df_final = new_df.merge(final_df,on=list_keys,how='left')
new_df_final.fillna(0,inplace=True)
new_df_final['ypred_mean_promo_new'] = new_df_final['ypred_mean_promo'] + new_df_final['mean_promo_reg_result']

In [19]:
new_df_final.head()

,Date,RDCKey,ProductKey,ypred_raw,ypred_mean_promo,salesForecast,mean_promo_reg_result,ypred_mean_promo_new
0,2018-04-01,3.0,193952,0.000785,0.000785,0.0,0.0,0.000785
1,2018-04-02,3.0,193952,0.000785,0.000785,0.0,0.0,0.000785
2,2018-04-03,3.0,193952,0.000785,0.000785,0.0,0.0,0.000785
3,2018-04-04,3.0,193952,0.000785,0.000785,0.0,0.0,0.000785
4,2018-04-05,3.0,193952,0.000785,0.000785,0.0,0.0,0.000785


In [6]:
new_df_final = pd.read_csv('./result_btup_lr/result_ensemble_2589.csv')

In [18]:
new_df_sku = new_df_final.groupby('ProductKey').sum().reset_index()
print "ensemble pred sum : %f"%(new_df_sku.ypred_mean_promo_new.sum())
print "raw pred sum :      %f"%(new_df_sku.ypred_raw.sum())
print "actual sum:         %f"%(new_df_sku.salesForecast.sum())

print "raw pred residual:      %f"%(np.sum(np.abs(new_df_sku.ypred_raw - new_df_sku.salesForecast)))
print "ensemble pred residual: %f"%(np.sum(np.abs(new_df_sku.ypred_mean_promo_new - new_df_sku.salesForecast)))

print "raw pred mape: %f"%(np.sum(np.abs(new_df_sku.ypred_raw - new_df_sku.salesForecast)) / new_df_sku.salesForecast.sum())
print "ensemble mape: %f"%(np.sum(np.abs(new_df_sku.ypred_mean_promo_new - new_df_sku.salesForecast)) / new_df_sku.salesForecast.sum())

ensemble pred sum : 39110.941565
raw pred sum :      40378.883483
actual sum:         37386.000000
raw pred residual:      16524.768977
ensemble pred residual: 15053.484354
raw pred mape: 0.442004
ensemble mape: 0.402650
